## Initialization

In [1]:
from datetime import datetime, timedelta
import arrow
import numpy as np
import os
import pandas as pd
import requests
import sys
import time

In [2]:
# with open('keenan_token.csv') as f:
#     secret_token = f.read().strip()

In [3]:
secret_token = 'YOUR SECRET TOKEN HERE'

In [4]:
# DO NOT USE THIS! USE kensho_external_graph_client

class KenshoGraphClient(object):
    """Basic implementation of the Kensho Graph API client"""

    def __init__(self, host, api_key, converter_func, retries=1):
        """Initialize the client"""
        self._host = host
        self._api_key = api_key
        self._retries = retries
        self._converter_func = converter_func

    def list_entity_classes(self):
        """List all available entity classes"""
        return self._get_json_or_raise('list_entity_classes')

    def list_timeline_types(self):
        """List all available entity classes"""
        return self._get_json_or_raise('list_timeline_types')

    def get_class_relationships(self, class_name):
        """Return all potential relationships of an entity of this class"""
        return self._get_json_or_raise(
            'get_class_relationships', class_name=class_name)

    def search_entities(self, class_name, search_string):
        """Search entities of the class_name that match search_string"""
        return self._get_json_or_raise(
            'search_entities', class_name=class_name, search_string=search_string)

    def list_entities_of_class(self, class_name):
        """List all entities of class"""
        return self._get_json_or_raise(
            'list_entities_of_class', class_name=class_name)

    def get_entity(self, entity_id):
        """Get entity by id"""
        return self._get_json_or_raise('get_entity', entity_id=entity_id)

    def get_related_entities(self, entity_id, relationship):
        """Get all entities related by one identified by entity_id by 'relationship'"""
        return self._get_json_or_raise(
            'get_related_entities', entity_id=entity_id, relationship=relationship)

    def get_timeline(self, timeline_id, start_date=None, end_date=None):
        """Get all events in a timeline. Optionally bound by start and end dates"""
        return self._get_json_or_raise(
            'get_timeline', timeline_id=timeline_id, start_date=start_date, end_date=end_date)

    def get_calendar(self, start_date, end_date):
        """Get all events happening in the interval [start_date, end_date)"""
        return self._get_json_or_raise('get_calendar', start_date=start_date, end_date=end_date)

    def get_ongoing_episodes(self, start_date, end_date):
        """Get all episodes ongoing in the interval [start_date, end_date)"""
        return self._get_json_or_raise(
            'get_ongoing_episodes', start_date=start_date, end_date=end_date)

    def translate_asset_id(self, id_type, asset_id):
        """Given an identification string for an asset return all known identifiers"""
        return self._get_json_or_raise('translate_asset_id', id_type=id_type, asset_id=asset_id)

    def _get_json_or_raise(self, function, **kwargs):
        """Get json from a given url. Uses get_with_retries underneath"""
        session = requests.Session()
        session.mount('https://', requests.adapters.HTTPAdapter(max_retries=self._retries))
        full_url = os.path.join(self._host, function)
        headers = {
            'Authorization': 'Token {}'.format(self._api_key),
            'Content-Type': 'application/json'
        }
        response = session.get(full_url, headers=headers, params=kwargs)
        response.raise_for_status()
        result_json = response.json()
        return self._converter_func(result_json)


def api_result_to_frame(api_result):
    """Convert API result into dataframe"""
    ordered_columns = [c['key'] for c in api_result['metadata']]
    # Create the dataframe
    df = pd.DataFrame(api_result['data'])
    existence_filtered_columns = [c for c in ordered_columns if c in df.columns]
    reordered_frame = df[existence_filtered_columns]

    for column in api_result['metadata']:
        if column['key'] not in reordered_frame.columns:
            continue
        if column['unit'] == 'DateTime' or column['unit'] == 'Date':
            reordered_frame[column['key']] = pd.to_datetime(reordered_frame[column['key']])
    return reordered_frame
    
def get_pandas_graph_client(host, api_key, retries=1):
    """Get a client that returns raw json"""
    return KenshoGraphClient(host, api_key, api_result_to_frame, retries=retries)


In [5]:
graph_client = get_pandas_graph_client('https://www.kensho.com/external/v1/', secret_token)

In [6]:
def ignore_timeline_types(df, timeline_types_to_ignore):
    if isinstance(timeline_types_to_ignore, str):
        raise ValueError('timeline_types_to_ignore may not be a string: {}'.format(timeline_types_to_ignore))
        
    indices = None
    for timeline_type in timeline_types_to_ignore:
        new_indices = df['timeline_type'] != timeline_type
        if indices is None:
            indices = new_indices
        else:
            indices &= new_indices
        
    return df[indices]

## List Functions

#### What kind of timelines are there in the Kensho Knowledge Graph?

In [7]:
graph_client.list_timeline_types()

,timeline_type
0,Academia
1,Activism
2,Administrative
3,Advisory
4,All Time High
5,Analyst Meetings
6,Analyst/Investor Day
7,Announcement of Interim Management Statement
8,Annual General Meeting
9,Appointments


#### What kind of entities are there in the Kensho Knowledge Graph? 

In [8]:
graph_client.list_entity_classes()

,class_name,can_list_entities
0,BankMeeting,False
1,CentralBank,True
2,Concept,True
3,Currency,True
4,Equity,False
5,Region,True
6,Sector,True
7,Timeline,False
8,TimelineType,True


#### Nice, so you have nine broad entity classes. How do I see all of the entities that comprise a single entity class? 

##### Example One: Currencies

In [9]:
graph_client.list_entities_of_class('Region').sort_values('entity_name').head(10)

,entity_name,entity_type,entity_id,entity_aliases,region_type,parent_name,parent_id
248,Afghanistan,Region,2f902415-6f14-4180-8e1b-e240b13b9736,"[Afghan, Islamic Republic of Afghanistan]",Country,Southern Asia,3df93e10-dfe1-4b8b-a8d4-d11e19b0c952
264,Africa,Region,6f12e428-1ea6-4b58-97b0-a8c9a5915d3f,[African],Region,"Europe, the Middle East and Africa",2a6f5acf-45a1-4189-bef4-2ca20b1347e9
327,Albania,Region,89891545-4da9-41dc-bee5-fe7117fcc4f3,"[Albanian, Republika e Shqipërisë, Shqipërisë,...",Country,Southern Europe,805534df-3823-4b83-9f36-bff2d08caf82
365,Algeria,Region,f06d884a-c362-4d93-8e63-dd3ccec06944,"[People’s Democratic Republic of Algeria, PDRA...",Country,Northern Africa,63716670-5806-45cf-9f31-e56e4725a794
26,American Samoa,Region,664e47af-cf2f-458d-8d9d-81fe46a7c4cb,[],Country,Polynesia,03755922-6cbb-4747-8b5e-0b7d023bed91
117,Americas,Region,ad7821cf-ed52-40ee-b991-b9147779ccef,[],Region,World,79bd87da-5cb9-45b7-8d1b-bdd3c21542ff
135,Andean Common Market,Region,0f4948b4-cb6e-4d43-8230-d514819d12df,[ANCOM],EconomicRegion,None,None
65,Andorra,Region,d331cc3e-a488-4e1b-a63d-5831b319fd0c,"[Principality of the Valleys of Andorra, Andor...",Country,Southern Europe,805534df-3823-4b83-9f36-bff2d08caf82
104,Angola,Region,3b45a4d9-e091-48db-aa42-04ed747be5a3,"[Republic of Angola, Angolan]",Country,Middle Africa,6d4af123-d3d7-482f-b2a1-8a189db31803
139,Anguilla,Region,7735309a-6456-422e-971f-05027fbd1c01,[Anguillan],Country,Caribbean,7844197e-7eba-428c-a7ce-543bea66cfdf


In [10]:
graph_client.list_entities_of_class('Concept').sort_values('entity_name')

,entity_name,entity_type,entity_id,entity_aliases
14,3D Printing,Concept,262737ff-18bc-492c-b460-2d96bd79f6c3,"[additive manufacturing, am]"
11,4K TV,Concept,83cad2d0-fed4-4d58-8922-5302dd74307a,"[4k resolution, 4k ultra high definition, 4k uhd]"
27,5G,Concept,dc93dd39-1b44-4278-8952-613b779b73fa,[]
1,Alcohol,Concept,78218550-cdb3-494d-b5de-c257e02a82b0,[]
10,Artificial Intelligence,Concept,7f4a32d8-9d6f-4087-8c42-3bed8e57a171,"[ai, machine intelligence, mi]"
17,Augmented Reality,Concept,4bbb8315-eabd-4a65-a0cc-4b1de43c58db,[ar]
6,Autonomous Vehicles,Concept,257edbf7-6185-4584-a5c4-91ac0b7bcbb3,"[driverless cars, self-driving cars, robot car..."
2,Cryptocurrency,Concept,e7ffc49f-3f96-4925-bf14-63d2438b1a8e,"[blockchain, crypto currency]"
8,Cybersecurity,Concept,5fa35db7-3437-4853-9802-4a6490b7f0ce,"[computer security, it security, cybersecurity]"
12,Data Storage,Concept,b17035bf-de7c-4b6b-941f-1badb581b985,"[data management, cloud storage]"


##### Example Two: Regions - How many countries / regions / economic regions are in the Kensho Knowledge Graph?

In [12]:
graph_client.list_entities_of_class('Region')['region_type'].value_counts()

Country           244
InformalRegion     68
Region             47
EconomicRegion     19
Name: region_type, dtype: int64

## Search Functions

#### Timelines

##### Example One: What Federal Open Markets Committee (FOMC) timelines are in the Kensho Knowledge Graph? 

In [13]:
graph_client.search_entities('Timeline', 'fomc')

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,Federal Open Markets Committee Meeting Minutes,Timeline,5d28d014-370c-4415-b6d3-3f1f54c0a363,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
1,Federal Open Markets Committee Meeting End,Timeline,0b2fa0d6-02e1-43db-a5c1-bfba327ad017,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
2,Federal Open Markets Committee Meeting Start,Timeline,eb5e5f23-3080-49a1-b5d8-df6f44c70bcd,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
3,Federal Open Markets Committee Meeting Statements,Timeline,bd2585f6-f5b9-4e0f-81a3-e1a77c7df4a2,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
4,Federal Open Markets Committee Leaves Interest...,Timeline,19bb1e6a-130a-4559-839d-ee3747f44238,[],Rate Actions,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
5,Federal Open Markets Committee Chair Speeches,Timeline,6a6e8724-97b0-4fc5-98c5-331727dead9e,[],Communications,[{'entity_id': '650505b8-ead1-45ee-93ba-7369a4...
6,Hawkish FOMC Meetings since 1990,Timeline,989127fc-4d5d-4b21-bbaa-5bd0d2cbae3e,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
7,Dovish FOMC Meetings since 1990,Timeline,ec15d342-625e-4007-bac9-c192af95fcb5,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
8,U.S. Federal Reserve Blackout Period End Dates,Timeline,7e558105-4bf6-41a9-9a5f-24dd76f4bf5b,[],Central Bank Speeches,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
9,U.S. Federal Reserve Blackout Period Start Dates,Timeline,d0261181-1287-41fe-9fff-a55626ebdea8,[],Central Bank Speeches,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...


In [18]:
#graph_client.search_entities('Timeline', 'labor')

In [19]:
#graph_client.search_entities('Timeline', 'nfl')

##### Example Two: I just saw a video of Jaysum Tatum breaking ankles at hoops camp. Anything on the NBA?  

In [20]:
graph_client.search_entities('Timeline', 'nba')[:10]

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,NBA Playoffs Begin,Timeline,5a676e90-f3d2-4efa-ae41-1920c51fe819,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
1,NBA Finals Start Dates,Timeline,c48f4ffb-419b-4705-ada3-8071511ad687,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
2,NBA Finals End Dates,Timeline,c87f4de0-ffd4-4663-b905-2e935cf02a7c,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
3,NBA 2K Video Game Releases,Timeline,6325a2bf-8b20-4cce-a806-a73cff01f3be,[],Product Releases,[{'entity_id': '47cb01e2-47d7-4b15-91e2-497ee8...
4,NBA All-Star Game,Timeline,01f67971-0fe5-450e-9215-dac10e2b1366,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
5,National Basketball Association (NBA) Draft,Timeline,7b80679f-165f-4af5-b4ce-c84e13fb74a5,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
6,Novabase (PT:NBA) Earnings Releases,Timeline,c3634253-485f-430a-a869-580b17c1ce73,[],Earnings,[{'entity_id': 'e8e36842-d63f-4d89-8523-f00539...
7,Major Multi-Console Video Game Releases in Nor...,Timeline,4a5b725b-4747-4dc1-872a-68d4fefb3b69,[],Product Releases,[{'entity_id': 'ec9ae1ac-33c6-4e44-b033-00377c...
8,Novabase Buyback Tranche Update,Timeline,b7139f4c-3cb0-4956-9b86-38baefefc601,[],Buyback Tranche Update,[{'entity_id': '25014370-ce33-4e5c-8bde-14facb...
9,Recent Key Company Developments,Timeline,cd7688e7-22b8-444a-867b-1fb38ba4978a,[],Corporate,[{'entity_id': '38a73500-3e05-4082-8cc8-8e0233...


In [21]:
timeline_df1 = graph_client.search_entities('Timeline', 'Weather')
timeline_df1['entity_name']

0     Global Cold Waves (as defined by U.S. National...
1     U.S. Weather and Climate Disasters Since 2017 ...
2     U.S. Weather and Climate Disasters Since 2017 ...
3                             Postponed SpaceX Launches
4             Major Global Oil Supply Chain Disruptions
5                  Dish Network Distribution Agreements
6                                 IBM Watson Milestones
7                              U.S. Retail Tax Holidays
8                             FedEx Service Disruptions
9     Unmanned Aerial Vehicle (Drone) Technology Mil...
10     Charter Communications Partnership Announcements
11                      Volkswagen Recall Announcements
12              Major U.S. Oil Supply Chain Disruptions
13                        Dish Network Channel Removals
14                     Red Hat Contract Wins since 2010
15    Negative Developments in Boeing 787 (Dreamline...
16    Dates of Major Cyber Attacks and Hacks in the ...
17        IBM Cloud Computing Partnership Announ

In [22]:
timeline_df1 = graph_client.search_entities('Timeline', 'TRIP')
timeline_df1['entity_name']

0        TripAdvisor, Inc. moves by 5% or more in a day
1     TripAdvisor, Inc. Company Conference Presentat...
2             Tripadvisor Inc. (TRIP) Earnings Releases
3     Sri Lankan President Maithripala Sirisena Inte...
4     Tripadvisor Inc. Corporate Guidance - New/Conf...
5               Tripadvisor Inc. Buyback Tranche Update
6       Tripadvisor Inc. Key Personnel Role Start Dates
7               TripAdvisor, Inc. Board Member Speeches
8                      TripAdvisor, Inc. Chair Speeches
9             Tripadvisor Inc. Board Member Start Dates
10           Changes to S&P 500 Constituents Since 2009
11    Liberty Tripadvisor Holdings Inc. (LTRPB) Earn...
12    Liberty Tripadvisor Holdings Inc. (LTRPA) Earn...
13        Tripadvisor Inc. Key Personnel Role End Dates
14    First Day of U.S. Presidential Trips to Saudi ...
15    Liberty TripAdvisor Holdings, Inc. President S...
16    Liberty TripAdvisor Holdings, Inc. Board Membe...
17      Liberty TripAdvisor Holdings, Inc. CEO S

In [23]:
timeline_df1 = graph_client.search_entities('Timeline', 'Weather')
#timeline_df1['entity_name']
timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']
#timeline_df1

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
1,U.S. Weather and Climate Disasters Since 2017 ...,Timeline,983f187d-ac0c-4beb-aa70-50316d29d346,[],Natural Disasters,[{'entity_id': '8fee99e4-9418-49b0-8a87-76449a...
2,U.S. Weather and Climate Disasters Since 2017 ...,Timeline,a0b14478-83fa-4e81-81b0-c2334d5cecd5,[],Natural Disasters,[{'entity_id': '8fee99e4-9418-49b0-8a87-76449a...


In [24]:
timeline_df1 = graph_client.search_entities('Timeline', 'Storm*')
#timeline_df1['entity_name']
#timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']

for i in timeline_df1['entity_name']:
    print(i)

Atlantic Hurricane and Tropical Storm Landfalls
Atlantic Hurricane and Tropical Storm Formations
Atlantic Hurricane and Tropical Storm Dissipations
East Pacific Hurricane and Tropical Storm Formations
East Pacific Hurricane and Tropical Storm Dissipations
West Pacific Typhoon and Tropical Storm Landfalls
West Pacific Typhoon and Tropical Storm Formations
West Pacific Typhoon and Tropical Storm Dissipations
Atlantic Hurricane and Tropical Storm Landfalls in the United States
East Pacific Hurricane and Tropical Storm Landfalls
West Pacific Typhoon and Tropical Storm Landfalls in Japan
East Pacific Hurricane and Tropical Storm Landfalls in Mexico
Snow Storms Over 4 Inches in a 24-Hour Period in the U.S. Northeast
Activision Blizzard, Inc. moves by 5% or more in a day
Activision Blizzard Inc (ATVI) Earnings Releases
Snow Storms Over 4 Inches in a 24-Hour Period in New York City
Storm Real Estate Buyback Tranche Update
Storm Resources Ltd. Corporate Guidance - New/Confirmed
United Kingdom (

In [25]:
timeline_df1 = graph_client.search_entities('Timeline', 'split')
#timeline_df1['entity_name']
#timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']

for i in timeline_df1['entity_name']:
    print(i)

M Split Corp. Preferred Dividend
TDb Split Corp. Preferred Dividend
TDb Split Corp. Dividend Affirmations
Dividend 15 Split Corp. Dividend Affirmations
Financial 15 Split Corp. Preferred Dividend
Mulvihill S Split Corp. Preferred Dividend
Brompton Split Banc Corp. Dividend Affirmations
Financial 15 Split Corp. Dividend Affirmations
Dividend 15 Split Corp. Preferred Dividend
Life & Banc Split Corp. Dividend Affirmations
Dividend Growth Split Corp. Dividend Affirmations
US Financial 15 Split Corp Preferred Dividend
Dividend 15 Split Corp. II Dividend Affirmations
New Commerce Split Fund Preferred Dividend
Dividend 15 Split Corp. II Preferred Dividend
Canadian Life Companies Split Corp. Preferred Dividend
North American Financial 15 Split Corp. Preferred Dividend
North American Financial 15 Split Corp. Dividend Affirmations
Canadian Life Companies Split Corp. Dividend Affirmations
Brompton Lifeco Split Corp. Dividend Affirmations


In [26]:
timeline_df1 = graph_client.search_entities('Timeline', 'Storm Landfalls')
#timeline_df1['entity_name']
#timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']

for i in timeline_df1['entity_name']:
    print(i)

Atlantic Hurricane and Tropical Storm Landfalls
West Pacific Typhoon and Tropical Storm Landfalls
Atlantic Hurricane and Tropical Storm Landfalls in the United States
East Pacific Hurricane and Tropical Storm Landfalls
West Pacific Typhoon and Tropical Storm Landfalls in Japan
East Pacific Hurricane and Tropical Storm Landfalls in Mexico
Atlantic Hurricane and Tropical Storm Landfalls in Mexico
West Pacific Typhoon and Tropical Storm Landfalls in China
United Kingdom (U.K.) and European Hurricane and Tropical Storm Landfalls
Activision Blizzard, Inc. moves by 5% or more in a day
West Pacific Typhoon and Tropical Storm Landfalls in the Philippines
Activision Blizzard Inc (ATVI) Earnings Releases
Storm Real Estate Buyback Tranche Update
Activision Blizzard Inc Corporate Guidance - New/Confirmed
Activision Blizzard, Inc. Company Conference Presentations
Storm Real Estate (NO:STOR) Earnings Releases
Activision Blizzard Inc Key Personnel Role Start Dates
Activision Blizzard Inc Buyback Tran

In [27]:
timeline_df1 = graph_client.search_entities('Timeline', 'Storm Landfalls')
#timeline_df1['entity_name']
#timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']

for i in timeline_df1['entity_name']:
    print(i)

Atlantic Hurricane and Tropical Storm Landfalls
West Pacific Typhoon and Tropical Storm Landfalls
Atlantic Hurricane and Tropical Storm Landfalls in the United States
East Pacific Hurricane and Tropical Storm Landfalls
West Pacific Typhoon and Tropical Storm Landfalls in Japan
East Pacific Hurricane and Tropical Storm Landfalls in Mexico
Atlantic Hurricane and Tropical Storm Landfalls in Mexico
West Pacific Typhoon and Tropical Storm Landfalls in China
United Kingdom (U.K.) and European Hurricane and Tropical Storm Landfalls
Activision Blizzard, Inc. moves by 5% or more in a day
West Pacific Typhoon and Tropical Storm Landfalls in the Philippines
Activision Blizzard Inc (ATVI) Earnings Releases
Storm Real Estate Buyback Tranche Update
Activision Blizzard Inc Corporate Guidance - New/Confirmed
Activision Blizzard, Inc. Company Conference Presentations
Storm Real Estate (NO:STOR) Earnings Releases
Activision Blizzard Inc Key Personnel Role Start Dates
Activision Blizzard Inc Buyback Tran

In [28]:
timeline_df1 = graph_client.search_entities('Timeline', 'Floods')
#timeline_df1['entity_name']
#timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']

for i in timeline_df1['entity_name']:
    print(i)

Major Floods in the U.S.
Flash Floods
Toyota Motor Discontinued Operations/Downsizings
Mergers and Acquisitions announcements by Fiserv
Significant Precipitation During 2014-2016 California Drought
Major Nigerian Oil Supply Chain Disruptions
Saudi Arabian Protests: Arab Spring to Present
Department of State (US) Secretary Speeches
Major U.S. Oil Supply Chain Disruptions
Janus Capital Group Inc Dividend Affirmations
Deloitte Touche Tohmatsu Company Conference Presentations
Major Global Oil Supply Chain Disruptions
Mosaic Company Discontinued Operations/Downsizings
Developments in Suspected Russian Cyber Attacks since 2007
Alterra Capital Holdings Limited Corporate Guidance - New/Confirmed
Staffing 360 Solutions Key Personnel Role Start Dates
MWH Global, Inc. Company Conference Presentations
Hillary Clinton Speeches
Data Processing & Outsourced Services Sector (code: 45102020) Mergers and Acquisitions
Emcore Corporation Discontinued Operations/Downsizings


In [29]:
timeline_df1 = graph_client.search_entities('Timeline', 'Earthquake')
#timeline_df1['entity_name']
#timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']

for i in timeline_df1['entity_name']:
    print(i)

World Earthquakes
Japan Earthquakes
Indonesia Earthquakes
BASIC Earthquakes
Pernambuco Earthquakes
Tonga Earthquakes
Caucasus Earthquakes
Mexico Earthquakes
Levant Earthquakes
VISTA Earthquakes
CIVETS Earthquakes
Oceania Earthquakes
PIGS Earthquakes
Asia Earthquakes
Portugal Earthquakes
Americas Earthquakes
NALA Earthquakes
Melanesia Earthquakes
Philippines Earthquakes
India Earthquakes


In [30]:
timeline_df1 = graph_client.search_entities('Timeline', 'storm category 4')
#timeline_df1['entity_name']
#timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']

for i in timeline_df1['entity_name']:
    print(i)

Activision Blizzard, Inc. moves by 5% or more in a day
Activision Blizzard Inc (ATVI) Earnings Releases
Storm Real Estate Buyback Tranche Update
Activision Blizzard Inc Corporate Guidance - New/Confirmed
Activision Blizzard, Inc. Company Conference Presentations
Storm Real Estate (NO:STOR) Earnings Releases
Activision Blizzard Inc Key Personnel Role Start Dates
Activision Blizzard Inc Buyback Tranche Update
American Homes 4 Rent Company Conference Presentations
Storm Real Estate Annual General Meeting
Activision Blizzard Inc Corporate Guidance - Raised
Activision Blizzard, Inc. Board Member Start Dates
American Homes 4 Rent (AMH) Earnings Releases
Activision Blizzard Inc Key Personnel Role End Dates
American Homes 4 Rent Preferred Dividend
American Homes 4 Rent Dividend Affirmations
Storm Real Estate Special/Extraordinary Shareholders Meeting
Storm Real Estate Buyback Transaction Announcements
Activision Blizzard, Inc. Board Member End Dates
American Homes 4 Rent Buyback Tranche Update

## Get Functions

#### You mentioned nodes and edges. What kind of relationships do you have for various equities?  

In [14]:
graph_client.get_class_relationships('Equity')

,relationship_id,related_class
0,EquityConcepts,Concept
1,Peers,Equity
2,EquitySuppliers,Equity
3,EquityConsumers,Equity
4,EquityTimelines,Timeline


## Entity Lookup

In [202]:
ticker_symbol = str('MSFT')
entity_info = graph_client.search_entities('Equity', ticker_symbol)
entity_info['entity_id']
entity_info

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Microsoft Corporation (MSFT),Equity,06826ee5-f5e2-4342-9bee-1a546db3b03b,"[Mikrosoft, Windows Corporation, Microsoft.com...",MSFT,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec


### Industry Involvement

In [203]:
entity_concepts = graph_client.get_related_entities(entity_info['entity_id'], 'EquityConcepts')
entity_concepts

,entity_name,entity_type,entity_id,entity_aliases
0,Video Gaming,Concept,56da58ef-10f5-4333-a7b0-f06c3fedbd33,"[electronic gaming, computer games, video games]"
1,Tablets,Concept,8a384f5c-4673-4c5e-a776-4780f4316161,[tablet computers]
2,Data Storage,Concept,b17035bf-de7c-4b6b-941f-1badb581b985,"[data management, cloud storage]"
3,Digital Assistants,Concept,c8114c49-9a5a-4ab1-b152-f03e99aa5731,[virtual assistant]
4,Cybersecurity,Concept,5fa35db7-3437-4853-9802-4a6490b7f0ce,"[computer security, it security, cybersecurity]"
5,Virtual Reality,Concept,2eaea590-2764-49ea-b64c-39c340e3b4f9,[vr]
6,Smart Homes,Concept,5ef8c951-f859-463a-9b69-e0cd55d1dace,[]
7,Smartphones,Concept,fa23a8bc-07e2-4bf2-970c-de68b7c3f431,"[smartphone, smart phone, smart phones]"
8,Social Networks,Concept,2805c9c2-8a5e-44ae-b7b7-7e61aa9d6e89,"[social networking site, social networking ser..."
9,Streaming Music,Concept,505b8e2a-bef4-4452-83c6-8635acb8462a,"[streaming music services, music streaming ser..."


In [204]:
entity_concepts.to_excel('{} Entity Concepts.xlsx'.format(ticker_symbol),sheet_name=ticker_symbol)

### Competitors 

In [205]:
entity_peers = graph_client.get_related_entities(entity_info['entity_id'], 'Peers')
entity_peers

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Apple Inc. (AAPL),Equity,f9d09c10-c6de-4e8f-9624-996774888bcc,"[Apple Computers, Apple comp, Apple Computer, ...",AAPL,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
1,"Amazon.com, Inc. (AMZN)",Equity,cfb73e32-4fa8-4701-898c-6c39279bbdb4,"[Amazon.com, Inc., Amazon company, Amazon Web ...",AMZN,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089
2,"salesforce.com, inc. (CRM)",Equity,5360a8c7-35c0-40ff-b819-20a9bfa83326,"[SalesForce, Salesforce.com]",CRM,Application Software,d173532e-b125-401d-87c1-c4076e748c69
3,"Facebook, Inc. (FB)",Equity,d8ce5287-c2c4-4c5a-8ec7-334b8213de4b,"[Thefacebook.com, The Facebook, Facebook, Inc....",FB,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
4,Alphabet Inc. (GOOG),Equity,37c53a09-309b-40bc-b165-90f0aac44366,"[Google, Alphabet Corp, Alphabet Inc, Abc.xyz,...",GOOG,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
5,Alphabet Inc. (GOOG.L),Equity,c03414b5-bf25-4f64-b5c8-ae50fc5b9e14,"[Google, Alphabet Corp, Alphabet Inc, Abc.xyz,...",GOOG.L,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
6,Altaba Inc. (AABA),Equity,6b7f93a8-cb9b-4358-a2a8-a1152377f4fc,"[Yahoo! Inc., Yahoo Alibaba Group, Yahoo, Yahoo!]",AABA,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089
7,Oracle Corporation (ORCL),Equity,b947ea63-eac3-4375-abee-333b1db74b85,"[Oracle Corp., Oracle Corp, Oracle Media Server]",ORCL,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec
8,"VMware, Inc. (VMW)",Equity,5569f0a7-3703-4506-8e13-48da4fb0eaa4,[],VMW,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec
9,The Sage Group plc (SGE),Equity,d8ca480b-49b0-44ef-a7c4-c7a4812b0a38,[],SGE,Application Software,d173532e-b125-401d-87c1-c4076e748c69


In [206]:
entity_peers.to_excel('{} Entity Peers.xlsx'.format(ticker_symbol),sheet_name=ticker_symbol)

### Supply Chain

In [207]:
entity_suppliers = graph_client.get_related_entities(entity_info['entity_id'], 'EquitySuppliers')
entity_suppliers.sort_values('entity_name')

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
79,A.L.D. Advanced Logistics Developments Ltd. (ALD),Equity,373160e8-b421-45bb-b42a-6f2ccd4fa450,[],ALD,IT Consulting & Other Services,8b091f43-5954-4b4c-8df6-357cac022547
36,"Activision Blizzard, Inc. (ATVI)",Equity,ce873cb2-9423-40a4-8fea-b2f0d756ab7c,"[Blizzard-Activision, Blizzard Activision, Act...",ATVI,Interactive Home Entertainment,5067324b-6872-4590-a1ad-957c5e377afe
83,Adda Corporation (3071),Equity,c6285e6d-1ee4-4d75-9539-e116e50366c5,[],3071,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
93,Addnode Group AB (publ) (ANOD B),Equity,bfddf6c1-5240-473c-a387-b55a4ece1326,[],ANOD B,IT Consulting & Other Services,8b091f43-5954-4b4c-8df6-357cac022547
39,"Advanced Micro Devices, Inc. (AMD)",Equity,778bcde1-adc4-4b6a-aa10-638d91821c7b,"[Advanced Micro Devices, Inc., Advanced Micro ...",AMD,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
145,"Akamai Technologies, Inc. (AKAM)",Equity,0a4da617-d3e4-4c19-a6f3-72221100e52b,"[Akamai, Akamai Technologies Inc]",AKAM,Internet Services & Infrastructure,a774f85b-a43e-4530-82c4-bf8af2cd4abe
92,"American Assets Trust, Inc. (AAT)",Equity,e650e36d-4466-4eb7-a3b8-1f57244793ca,[],AAT,Diversified REITs,b2a2229a-7341-4bef-8437-c9d08d83e83d
86,"American Software, Inc. (AMSW.A)",Equity,3c3b3f7c-07ea-4335-8c3b-88f0dd22b454,[],AMSW.A,Application Software,d173532e-b125-401d-87c1-c4076e748c69
147,"Applied Optoelectronics, Inc. (AAOI)",Equity,69153e74-6dd3-41c8-b1da-de538968e2e8,[],AAOI,Communications Equipment,aab0be3f-7ae4-4680-9e8b-811d8fabcf93
21,"Arista Networks, Inc. (ANET)",Equity,158abe3a-5b38-47f5-bb00-523944cf4328,[],ANET,Communications Equipment,aab0be3f-7ae4-4680-9e8b-811d8fabcf93


In [208]:
entity_suppliers.to_excel('{} Entity Suppliers.xlsx'.format(ticker_symbol),sheet_name=ticker_symbol)

##### Entity Suppliers

In [209]:
entity_suppliers['sector_name'].value_counts()

Application Software                          22
IT Consulting & Other Services                15
Advertising                                   11
Semiconductors                                 8
Systems Software                               8
Communications Equipment                       8
Technology Hardware, Storage & Peripherals     8
Electronic Components                          7
Electronic Equipment & Instruments             7
Interactive Home Entertainment                 6
Internet Services & Infrastructure             4
Human Resource & Employment Services           4
Electrical Components & Equipment              3
Movies & Entertainment                         3
Diversified Capital Markets                    2
Real Estate Operating Companies                2
Paper Packaging                                2
Electronic Manufacturing Services              2
Interactive Media & Services                   2
Construction & Engineering                     2
Automobile Manufactu

### Consumers

In [211]:
entity_consumers = graph_client.get_related_entities(entity_info['entity_id'], 'EquityConsumers')
#entity_consumers.sort_values('entity_name').head()
entity_consumers

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Verizon Communications Inc. (VZ),Equity,0f154c5b-90a2-40fc-89a0-c9fce7d05676,"[Verizon, Verizon VoiceWing, VZN, Verizon.net,...",VZ,Integrated Telecommunication Services,3153749b-509c-48ba-9705-eafa1d23d71a
1,QAD Inc. (QADB),Equity,a62aef90-7f90-4fac-bc4e-20de7f740606,[],QADB,Application Software,d173532e-b125-401d-87c1-c4076e748c69
2,QAD Inc. (QADA),Equity,5e6a2c49-45cd-48e7-a733-ecd65c01fe33,[],QADA,Application Software,d173532e-b125-401d-87c1-c4076e748c69
3,"Best Buy Co., Inc. (BBY)",Equity,76d50abb-86dd-4931-abc0-4594bf84b4d8,"[Best Buy Co., Inc., Best Buy Co. Inc., Best B...",BBY,Computer & Electronics Retail,fa406110-e163-4bee-a47f-3b38f094a948
4,"Tangoe, Inc. (TNGO)",Equity,8f48b1fa-6668-43e4-931e-0fb746390d56,[],TNGO,Application Software,d173532e-b125-401d-87c1-c4076e748c69
5,Cellcom Israel Ltd. (CEL),Equity,30b1be84-0cd2-4ad1-82f4-8c8d484c08e8,[],CEL,Wireless Telecommunication Services,558a4212-6576-4fe6-b121-21e0eb91edb9
6,Cellcom Israel Ltd. (CEL),Equity,72cf95f6-0c9d-47ba-8b40-2ca3dede4884,[],CEL,Wireless Telecommunication Services,558a4212-6576-4fe6-b121-21e0eb91edb9
7,"Fluent, Inc. (FLNT)",Equity,469c388b-b982-4530-95dc-899036d524b6,[],FLNT,Advertising,f83e6b06-b017-4e63-b9dc-9cd5eebf37b2
8,"Web.com Group, Inc. (WEB)",Equity,de7e39d9-85d6-41fc-a735-1d5d889d984a,[],WEB,Internet Services & Infrastructure,a774f85b-a43e-4530-82c4-bf8af2cd4abe
9,Nielsen Holdings plc (NLSN),Equity,5f99cc48-26cc-4d73-9f74-399871dbc965,"[A.C. Nielsen Co., The Nielsen Company, Nielse...",NLSN,Research & Consulting Services,c5a762e2-e4ee-4fbb-b57d-219ba64fe2b1


In [212]:
entity_consumers.to_excel('{} Entity Consumers.xlsx'.format(ticker_symbol),sheet_name=ticker_symbol)

# Get event timelines for a single entity

In [223]:
entity_info = graph_client.search_entities('Equity', 'MSFT')
#entity_info['entity_id']
#entity_info
entity_info['entity_id'] = entity_info['entity_id'][0]

In [224]:
%%time
entity_timeline = pd.DataFrame(graph_client.get_related_entities(entity_info['entity_id'],'EquityTimelines'))

Wall time: 368 ms


In [225]:
entity_timeline.shape

(76, 6)

In [226]:
entity_timeline.head()

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,Microsoft Corporation (MSFT) Earnings Releases,Timeline,b4897d0f-f99c-4759-bcf6-486dc765db8f,[],Earnings,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...
1,Microsoft Corporation moves by 5% or more in a...,Timeline,c9dcbb6f-59e7-4752-bd9b-3d4c1d051928,[],Technical Indicators,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...
2,Microsoft Corporation Corporate Guidance - Low...,Timeline,098dbe9f-0339-48e5-9be9-b51bd30454bf,[],Corporate Guidance - Lowered,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...
3,Microsoft Corporation Corporate Guidance - Raised,Timeline,53117257-02cc-4deb-bca6-69209316b2bf,[],Corporate Guidance - Raised,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...
4,Microsoft Corporation Corporate Guidance - New...,Timeline,692b1c99-3bc7-46e5-aa30-8231fcc711d1,[],Corporate Guidance - New/Confirmed,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...


In [217]:
entity_symbol = entity_info['ticker_name'][0]
entity_timeline.to_excel('{} Event Timeline.xlsx'.format(entity_symbol),sheet_name=entity_symbol)

# Event timelines by date range and entity

In [29]:
# We limit get_calendar pulls to ten days for throttling reasons - this is how I sidestep it: 

start_dates = []
end_dates = []

start = datetime.today() - timedelta(365) # Using 63 days as a proxy for a quarter - pass in whatever lookback period you need  

while start < datetime.today(): # While loop to prevent the function from pulling all future events already loaded into the Graph 
    start_dates.append(start.strftime('%Y-%m-%d'))
    
    end = start + timedelta(10)
    end_dates.append(end.strftime('%Y-%m-%d'))
    
    start = end + timedelta(1)

In [30]:
%%time
trailing_quarter_events = pd.DataFrame()

for start, end in zip(start_dates, end_dates):
    temp_frame = graph_client.get_calendar(start, end)
    trailing_quarter_events = pd.concat((trailing_quarter_events, temp_frame), axis=0)

C:\Users\zanderl\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Wall time: 12min 21s


In [31]:
ttypes = graph_client.list_entities_of_class('TimelineType')

# Instantiating a dict to streamline id searching and saving
ttypes_dict = dict(zip(ttypes['entity_name'], ttypes['entity_id'])) # ttypes_dict includes all 212 TimelineTypes in the Graph 

In [32]:
trailing_quarter_events.shape
#trailing_quarter_events.head()

(353303, 10)

In [38]:
trailing_quarter_events.head()

,event_date,event_date_precision,event_name,event_description,timeline_id,timeline_name,timeline_type,related_entities,primary_provenance,provenance,entity_bool
0,2018-02-11,YYYY-MM-DD,Gran Tierra Energy Inc. Presents at Credit Sui...,Gran Tierra Energy Inc. Presents at Credit Sui...,0089b54f-dfda-4d5a-a131-752faebaae9a,Gran Tierra Energy Inc. Company Conference Pre...,Company Conference Presentations,[{'entity_id': '08278a24-f73a-4945-ac4c-dff0f4...,[],[],0
1,2018-02-11,YYYY-MM-DD,"Guangxi Guidong Electric Power Co., LTD. annou...",,014b4dac-fd64-4316-9789-5168932588b2,Mergers and Acquisitions by Companies in China,M&A,[{'entity_id': '03b45cc0-f14a-4898-968b-8b8f60...,[],[],0
2,2018-02-11,YYYY-MM-DD,Cars.com Inc. Presents at IAB Annual Leadershi...,Cars.com Inc. Presents at IAB Annual Leadershi...,0268f5d2-9588-4505-8690-0b589c863519,Cars.com Inc. Company Conference Presentations,Company Conference Presentations,[{'entity_id': '91c30712-5217-401b-9eee-97ec6c...,[],[],0
3,2018-02-11,YYYY-MM-DD,Bettina Whyte begins tenure as a Board Member ...,,0bb0eb6e-acff-4c88-b8b6-0a9c8404d661,WestRock Company Board Member Start Dates,Personnel,[{'entity_id': '2f27f4dc-b065-4e48-b9ef-86d171...,[],[],0
4,2018-02-11,YYYY-MM-DD,Alphabet Inc. Presents at Annual PACnet commun...,Alphabet Inc. Presents at Annual PACnet commun...,142b4a0e-bb52-4695-90c0-40d4c76f5053,Alphabet Inc. Company Conference Presentations,Company Conference Presentations,[{'entity_id': '37c53a09-309b-40bc-b165-90f0aa...,[],[],1


In [227]:
entity_info = graph_client.search_entities('Equity', 'MSFT')
#entity_info['entity_id']
#entity_info
entity_info['entity_id'] = entity_info['entity_id'][0]

In [228]:
%%time

entity_bool = []

for entity_list in trailing_quarter_events['related_entities']:
    related_to_entity = 0
    
    for entity in entity_list:
        if entity['entity_id'] == entity_info['entity_id'].all():
            related_to_entity = 1
        
    entity_bool.append(related_to_entity)

trailing_quarter_events['entity_bool'] = entity_bool

entity_calendar = pd.DataFrame(trailing_quarter_events[(trailing_quarter_events['entity_bool']==1)])

Wall time: 16.1 s


In [229]:
entity_calendar.shape

(584, 11)

In [230]:
entity_calendar

,event_date,event_date_precision,event_name,event_description,timeline_id,timeline_name,timeline_type,related_entities,primary_provenance,provenance,entity_bool
458,2018-02-12 00:00:00,YYYY-MM-DD,Microsoft Corporation Presents at ARC Industry...,Microsoft Corporation Presents at ARC Industry...,e1556eeb-2ca7-4945-bc63-698aea89b460,Microsoft Corporation Company Conference Prese...,Company Conference Presentations,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...,[],[],1
1690,2018-02-13 00:00:00,YYYY-MM-DD,IBM sues its former Human Resources Vice Presi...,International Business Machines Corp sues its ...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...,[],[https://www.reuters.com/article/us-ibm-micros...,1
1691,2018-02-13 00:00:00,YYYY-MM-DD,Microsoft announces that Chalkup along with Ch...,"Microsoft announces that Chalkup’s co-founder,...",cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...,[https://educationblog.microsoft.com/2018/02/c...,[https://techcrunch.com/2018/02/13/microsoft-a...,1
1692,2018-02-13 00:00:00,YYYY-MM-DD,Microsoft announces that Chalkup along with Ch...,"Microsoft announces that Chalkup’s co-founder,...",cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...,[https://educationblog.microsoft.com/2018/02/c...,[https://techcrunch.com/2018/02/13/microsoft-a...,1
2482,2018-02-13 16:00:00,YYYY-MM-DDTHH:mmZ,The 2018 Goldman Sachs Technology and Internet...,,7fc43907-ea7e-4836-84b8-4bf981cfc7d5,Goldman Sachs Technology and Internet Conference,Industry Conferences,[{'entity_id': '03e7519b-cd37-4b57-8aa4-1fb582...,[],[https://www.verint.com/press-room/2018-press-...,1
4558,2018-02-14 21:15:00,YYYY-MM-DDTHH:mmZ,Microsoft Corporation Presents at Goldman Sach...,Microsoft Corporation Presents at Goldman Sach...,e1556eeb-2ca7-4945-bc63-698aea89b460,Microsoft Corporation Company Conference Prese...,Company Conference Presentations,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...,[],[],1
10114,2018-02-16 16:45:00,YYYY-MM-DDTHH:mmZ,Microsoft Corporation Presents at 2018 Munich ...,Microsoft Corporation Presents at 2018 Munich ...,e1556eeb-2ca7-4945-bc63-698aea89b460,Microsoft Corporation Company Conference Prese...,Company Conference Presentations,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...,[],[],1
11465,2018-02-20 14:45:00,YYYY-MM-DDTHH:mmZ,Microsoft Corporation Presents at NYC Licensin...,Microsoft Corporation Presents at NYC Licensin...,e1556eeb-2ca7-4945-bc63-698aea89b460,Microsoft Corporation Company Conference Prese...,Company Conference Presentations,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...,[],[],1
11598,2018-02-20 21:00:00,YYYY-MM-DDTHH:mmZ,Microsoft Corporation Presents at HITRUST Thir...,Microsoft Corporation Presents at HITRUST Thir...,e1556eeb-2ca7-4945-bc63-698aea89b460,Microsoft Corporation Company Conference Prese...,Company Conference Presentations,[{'entity_id': '06826ee5-f5e2-4342-9bee-1a546d...,[],[],1
101,2018-02-22 00:00:00,YYYY-MM-DD,Intel announces collaboration with Hewlett Pac...,Intel announces it is collaborating with Micro...,43b37235-f577-4d2f-b394-301ffdcbcdff,Intel Corporate Partnership Announcements,Partnerships,[{'entity_id': '9d110065-1215-469b-81e0-1e626a...,[https://newsroom.intel.com/editorials/intel-s...,[https://www.theverge.com/2018/2/22/17031770/i...,1


In [231]:
entity_symbol = entity_info['ticker_name'][0]
entity_calendar.to_excel('{} Entity Calendar.xlsx'.format(entity_symbol),sheet_name=entity_symbol)

In [222]:
#%%time
#next_week = graph_client.get_calendar('2018-08-23', '2018-09-01')
#next_week.head(20)

In [45]:
#next_week.shape

#### Let's narrow that down.. What earnings-related events are coming up?  

In [ ]:
next_week[next_week['event_name'].str.lower().str.contains('earning')].sort_values('event_name')[['event_date', 'event_date_precision', 'event_name']].head(10)

#### What about conferences? 

In [ ]:
next_week[next_week['event_name'].str.lower().str.contains('conference')].sort_values('event_name')[['event_date', 'event_date_precision', 'event_name']].head(10)

## Concepts (i.e., Thematic Search)

##### I mentioned you can search a company and see what concepts/industries/themes it is involved with. What if you want to approach this in reverse? The Kensho Knowledge Graph is recursive, meaning you can also see a list of companies involved in said a concept/industry/theme (e.g., wearables). 

In [130]:
graph_client.list_entities_of_class('Concept')

,entity_name,entity_type,entity_id,entity_aliases
0,Drones,Concept,d1d39e88-4673-493e-8362-db486006ce1e,"[unmanned aircraft systems, unmanned aerial ve..."
1,Alcohol,Concept,78218550-cdb3-494d-b5de-c257e02a82b0,[]
2,Cryptocurrency,Concept,e7ffc49f-3f96-4925-bf14-63d2438b1a8e,"[blockchain, crypto currency]"
3,Streaming Video,Concept,45b36735-15db-4b45-b2c6-8739f7365377,"[streaming video services, video sharing websi..."
4,Digital Payments,Concept,96d1bc76-938e-41a2-ac17-be05a71c98ae,"[mobile payments, e-commerce payment system, o..."
5,Wireless Charging,Concept,12552d8a-cee7-4d21-9b97-94bbc74d37d6,"[inductive charging, cordless charging]"
6,Autonomous Vehicles,Concept,257edbf7-6185-4584-a5c4-91ac0b7bcbb3,"[driverless cars, self-driving cars, robot car..."
7,E-Commerce Platforms,Concept,5a090fc8-38e8-4811-8827-da54cf71ed8a,[]
8,Cybersecurity,Concept,5fa35db7-3437-4853-9802-4a6490b7f0ce,"[computer security, it security, cybersecurity]"
9,Television,Concept,ff4d38f7-406a-4184-b6f3-498a0aeb2d1f,[]


#### What companies are involved in wearables? 

In [131]:
graph_client.get_related_entities('1f0667ed-3009-463a-9301-bfe158f863a8', 'ConceptEquities')

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Apple Inc. (AAPL),Equity,f9d09c10-c6de-4e8f-9624-996774888bcc,"[Apple Computers, Apple comp, Apple Computer, ...",AAPL,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
1,Garmin Ltd. (GRMN),Equity,67958864-0c6c-4fef-851f-ff43fd196212,"[Garmin International, Inc., Garmin Internatio...",GRMN,Consumer Electronics,fc9bd158-ebba-433a-88ce-ed389cbe1c84
2,"Skyworks Solutions, Inc. (SWKS)",Equity,941ef5a5-19cd-436d-a8df-18af60742539,"[Skyworks Solutions Inc., Skyworks Solutions I...",SWKS,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
3,Amphenol Corporation (APH),Equity,e9b43a37-9718-4f2b-bee4-6a7747e28f7f,[],APH,Electronic Components,19a722a6-e905-4656-945c-1d5022b06757
4,Michael Kors Holdings Limited (CPRI),Equity,18a3abf4-6b89-4c09-b1d0-df541b4f4b8b,[Michael Kors Holdings],CPRI,"Apparel, Accessories & Luxury Goods",56693b1f-f831-4de8-9e1a-da144ea29053
5,QUALCOMM Incorporated (QCOM),Equity,62ad3190-eb46-4824-bbe0-5e1ca3657fea,"[QUALCOMM, Qualcomm Incorporated]",QCOM,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
6,Western Digital Corporation (WDC),Equity,ee1bda7a-af07-4be8-89de-a46335b7934e,[Western Digital],WDC,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
7,"Analog Devices, Inc. (ADI)",Equity,39182bbd-5bf6-4bc2-82df-99b28521229a,"[Analog Devices, Inc., Analog Devices Inc, Ana...",ADI,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
8,Broadcom Inc. (AVGO),Equity,130b60d9-7078-4079-ba70-dc4b818aeaec,[],AVGO,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
9,"Qorvo, Inc. (QRVO)",Equity,0ebbdfd6-d8e0-4a55-b0fb-b66a1498832c,[Qorvo],QRVO,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b


In [34]:
graph_client.get_related_entities('9985d1d3-53bd-4879-bd89-5eef0f83c50d', 'ConceptEquities')

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Aptiv PLC (APTV),Equity,76264a6c-11cf-4bec-bbca-fb292b384f5b,"[Delphi Corporation, Delphi Automotive Systems...",APTV,Auto Parts & Equipment,ef94be36-634a-4247-8e23-1c5d4af2a128
1,BorgWarner Inc. (BWA),Equity,34076650-a7cb-487c-b5b5-32bdb29fcc16,"[BorgWarner, Borgwarner Inc, Borg-Warner, Borg...",BWA,Auto Parts & Equipment,ef94be36-634a-4247-8e23-1c5d4af2a128
2,General Motors Company (GM),Equity,9adb4f1a-5904-4e11-a83e-5401b76f6c06,"[General Motors, General Motors Company LLC, G...",GM,Automobile Manufacturers,d8924a95-cac6-42d0-93e1-d696cfcd2621
3,"American Axle & Manufacturing Holdings, Inc. (...",Equity,e1e714bf-a71e-44ac-82e0-7a061abe387a,[],AXL,Auto Parts & Equipment,ef94be36-634a-4247-8e23-1c5d4af2a128
4,"AeroVironment, Inc. (AVAV)",Equity,8174abd9-1441-413d-9f8a-5b49fcdd0a63,[],AVAV,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
5,FMC Corporation (FMC),Equity,340e8890-fe33-4582-bf04-37f4c57404c4,"[Food Machinery and Chemical Corporation, FMC ...",FMC,Fertilizers & Agricultural Chemicals,f8a7df4a-4249-4d56-8b78-7d2506ca3d0a
6,Ford Motor Company (F),Equity,51cdd4cc-4211-4c25-a11b-886ff53b85ec,"[Ford Motor Works, Ford Motor, Ford, Ford Moto...",F,Automobile Manufacturers,d8924a95-cac6-42d0-93e1-d696cfcd2621
7,"Power Integrations, Inc. (POWI)",Equity,2afe2ef1-1ab9-4545-9036-a6e4bc936d84,[],POWI,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
8,Mitsubishi Motors Corporation (7211),Equity,cf8074b3-88e3-4e87-8d5b-9c6b16629017,[],7211,Automobile Manufacturers,d8924a95-cac6-42d0-93e1-d696cfcd2621
9,Renault SA (RNO),Equity,8fef2278-0a58-482d-8314-917909d8f774,[],RNO,Automobile Manufacturers,d8924a95-cac6-42d0-93e1-d696cfcd2621


In [ ]:
graph_client.get_related_entities('1f0667ed-3009-463a-9301-bfe158f863a8', 'ConceptEquities')

Companies involved with the Streaming Music concept

In [132]:
graph_client.get_related_entities('505b8e2a-bef4-4452-83c6-8635acb8462a', 'ConceptEquities')

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Alphabet Inc. (GOOG),Equity,37c53a09-309b-40bc-b165-90f0aac44366,"[Google, Alphabet Corp, Alphabet Inc, Abc.xyz,...",GOOG,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
1,Alphabet Inc. (GOOG.L),Equity,c03414b5-bf25-4f64-b5c8-ae50fc5b9e14,"[Google, Alphabet Corp, Alphabet Inc, Abc.xyz,...",GOOG.L,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
2,Apple Inc. (AAPL),Equity,f9d09c10-c6de-4e8f-9624-996774888bcc,"[Apple Computers, Apple comp, Apple Computer, ...",AAPL,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
3,"Amazon.com, Inc. (AMZN)",Equity,cfb73e32-4fa8-4701-898c-6c39279bbdb4,"[Amazon.com, Inc., Amazon company, Amazon Web ...",AMZN,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089
4,Pandora A/S (PNDORA),Equity,7f08aea6-c7f5-463c-a395-9ce00ebed77a,[],PNDORA,"Apparel, Accessories & Luxury Goods",56693b1f-f831-4de8-9e1a-da144ea29053
5,Microsoft Corporation (MSFT),Equity,06826ee5-f5e2-4342-9bee-1a546db3b03b,"[Mikrosoft, Windows Corporation, Microsoft.com...",MSFT,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec


### Companies within the Cybersecurity conecpt

In [232]:
concept_constituents = graph_client.get_related_entities('5fa35db7-3437-4853-9802-4a6490b7f0ce', 'ConceptEquities')
concept_constituents

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Symantec Corporation (SYMC),Equity,f2743ccf-e05c-4884-b8d4-7aa8bfa1ac49,[Symantec Corp.],SYMC,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec
1,Raytheon Company (RTN),Equity,155b7dea-f670-4c0a-87f2-ce56bf68d9c8,"[Raytheon Corp, Raytheon Corporation, The Rayt...",RTN,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
2,"Akamai Technologies, Inc. (AKAM)",Equity,0a4da617-d3e4-4c19-a6f3-72221100e52b,"[Akamai, Akamai Technologies Inc]",AKAM,Internet Services & Infrastructure,a774f85b-a43e-4530-82c4-bf8af2cd4abe
3,"Motorola Solutions, Inc. (MSI)",Equity,535bc615-9693-4ace-bc0c-9c54cfc04275,"[Motorola Solutions, Inc., Motorola Solutions ...",MSI,Communications Equipment,aab0be3f-7ae4-4680-9e8b-811d8fabcf93
4,"F5 Networks, Inc. (FFIV)",Equity,d017caaf-3b4d-4920-9e1a-58a46d6ba20e,[F5 Networks Inc],FFIV,Communications Equipment,aab0be3f-7ae4-4680-9e8b-811d8fabcf93
5,"Citrix Systems, Inc. (CTXS)",Equity,702350dc-6ef8-4310-a5ab-a7aa5e7af188,"[Citrix, Citrix Systems, Inc.]",CTXS,Application Software,d173532e-b125-401d-87c1-c4076e748c69
6,"Cisco Systems, Inc. (CSCO)",Equity,a672039a-480d-4176-8270-9b17aa0ef9b7,"[Cisco Systems Incorporated, Cisco, Cisco Syst...",CSCO,Communications Equipment,aab0be3f-7ae4-4680-9e8b-811d8fabcf93
7,"Level 3 Parent, LLC (LVLT)",Equity,623b20a9-deeb-4daa-9da6-95e3927c9350,"[Level 3 Communications Incorporated, Level3.n...",LVLT,Alternative Carriers,ef845e3f-0f06-4f6e-996e-2729b1f657cd
8,"VeriSign, Inc. (VRSN)",Equity,cfe5e2bb-37dc-4344-aab0-efa14442cf99,"[VeriSign Communications Services, VeriSign In...",VRSN,Internet Services & Infrastructure,a774f85b-a43e-4530-82c4-bf8af2cd4abe
9,CA Technologies (CA),Equity,6a6e6e8a-69a8-41ad-9c8f-b282587d3ac9,"[Computer Associates, CA, Inc, Computer Associ...",CA,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec


In [234]:
concept_name = 'Cybersecurity'
concept_constituents.to_excel('{} Concept Constituents.xlsx'.format(concept_name),sheet_name=concept_name)

### Drones concept constituents

In [233]:
concept_constituents = graph_client.get_related_entities('d1d39e88-4673-493e-8362-db486006ce1e', 'ConceptEquities')
concept_constituents

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Textron Inc. (TXT),Equity,5e50d559-8942-476a-a02f-29fa1f8d0130,"[Textron Incorporated, HR Textron, Textron Cor...",TXT,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
1,The Boeing Company (BA),Equity,39cca9d3-52a3-461e-b288-0dfb320c407e,"[Boeing Corporation, The Boeing Co., The Boein...",BA,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
2,TransDigm Group Incorporated (TDG),Equity,7b17a315-4fef-4406-b29a-36e357c86867,"[TransDigm, Transdigm Group, TransDigm Incorpo...",TDG,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
3,Northrop Grumman Corporation (NOC),Equity,f7a0749c-1547-4126-8416-d0a9eb32ae48,"[Northrup Grumman Corporation, Northrop-Grumma...",NOC,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
4,Raytheon Company (RTN),Equity,155b7dea-f670-4c0a-87f2-ce56bf68d9c8,"[Raytheon Corp, Raytheon Corporation, The Rayt...",RTN,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
5,"Rockwell Collins, Inc. (COL)",Equity,d5fabc78-6e9f-4d5f-94d9-42d43ef81198,"[Rockwell Collins Inc, Collins Radio Co., Coll...",COL,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
6,Parker-Hannifin Corporation (PH),Equity,b5ea17fd-bc59-4124-b16b-2ff73b886c75,"[Parker Hannifin Corp, Parker Hannifin Corpora...",PH,Industrial Machinery,befb8d13-cfaf-4061-9f9d-a9d59a032285
7,Lockheed Martin Corporation (LMT),Equity,b7ffc2ea-1f3e-4eaf-b229-bbccdc5fd673,"[Lockheed martin, Lockheed Martin Corp., LockM...",LMT,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a
8,"FLIR Systems, Inc. (FLIR)",Equity,f3bfcba0-61b5-43db-bfea-0f6a4013edce,"[FLIR Systems Inc., Flir system]",FLIR,Electronic Equipment & Instruments,ad8a5f5e-e047-41a4-b3ea-e7662c45bdd2
9,Harris Corporation (HRS),Equity,7f17ebdf-7611-4dc5-888d-593e2a790b93,"[Harris Corp, Harris Semiconductor, Harris Corp.]",HRS,Aerospace & Defense,fe01c187-9cd1-4754-a26f-a772d5a51b0a


In [235]:
concept_name = 'Drones'
concept_constituents.to_excel('{} Concept Constituents.xlsx'.format(concept_name),sheet_name=concept_name)

### Video gaming concept constituents

In [236]:
concept_constituents = graph_client.get_related_entities('56da58ef-10f5-4333-a7b0-f06c3fedbd33', 'ConceptEquities')
concept_constituents

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,GameStop Corp. (GME),Equity,a995b500-e7df-47cb-8f94-30d3ddd797d2,[],GME,Computer & Electronics Retail,fa406110-e163-4bee-a47f-3b38f094a948
1,Microsoft Corporation (MSFT),Equity,06826ee5-f5e2-4342-9bee-1a546db3b03b,"[Mikrosoft, Windows Corporation, Microsoft.com...",MSFT,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec
2,Electronic Arts Inc. (EA),Equity,3ef8d9aa-e83e-4f2d-ada0-0bd3e3bf445a,"[Electronic Arts Inc, Eagames, EA Games, Elect...",EA,Interactive Home Entertainment,5067324b-6872-4590-a1ad-957c5e377afe
3,NVIDIA Corporation (NVDA),Equity,7f4d2aeb-8f98-4aea-8799-0bb122ea4e7d,"[NVIDIA Corp., NVDIA]",NVDA,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
4,"Activision Blizzard, Inc. (ATVI)",Equity,ce873cb2-9423-40a4-8fea-b2f0d756ab7c,"[Blizzard-Activision, Blizzard Activision, Act...",ATVI,Interactive Home Entertainment,5067324b-6872-4590-a1ad-957c5e377afe
5,Konami Holdings Corporation (9766),Equity,be23c017-265b-4d40-ad9c-0c4f9045f566,[],9766,Interactive Home Entertainment,5067324b-6872-4590-a1ad-957c5e377afe
6,"Capcom Co., Ltd. (9697)",Equity,84d249a8-34d0-42e0-addc-f7d6bfb9f53c,[],9697,Interactive Home Entertainment,5067324b-6872-4590-a1ad-957c5e377afe
7,Ubisoft Entertainment SA (UBI),Equity,031c2fdb-3631-4b41-9005-ff1e49422b38,[],UBI,Interactive Home Entertainment,5067324b-6872-4590-a1ad-957c5e377afe
8,Focus Home Interactive Société anonyme (ALFOC),Equity,4c95d545-cb35-4a52-9fb7-768cc4ad1d08,[],ALFOC,Interactive Home Entertainment,5067324b-6872-4590-a1ad-957c5e377afe
9,"GungHo Online Entertainment, Inc. (3765)",Equity,a36de2c5-6fff-482d-bc12-9c0388ec5335,[],3765,Interactive Home Entertainment,5067324b-6872-4590-a1ad-957c5e377afe


In [237]:
concept_name = 'Video Gaming'
concept_constituents.to_excel('{} Concept Constituents.xlsx'.format(concept_name),sheet_name=concept_name)